In [1]:
import json

In [2]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [3]:
import import_ipynb
from aux import utils
from aux import nlp
from aux import relation_extraction
from aux import defs
import preparation
import rule_base

importing Jupyter notebook from /Users/YK/mt/project/aux/utils.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/nlp.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/relation_extraction.ipynb
importing Jupyter notebook from /Users/YK/mt/project/aux/defs.ipynb
importing Jupyter notebook from preparation.ipynb
importing Jupyter notebook from rule_base.ipynb


In [4]:
class RuleExplanation03(rule_base.Rule):
    name = "explanation_03"
    relation_type = "Explanation"
    reasons = {
        "SN_CONTAINS_IN_FACT_ETC": 
            defs.Reason(
                1, 
                "Satellite's nucleus contains " 
                "'in fact' / 'as a matter of fact' / 'actually' / 'indeed' / 'also'."),
        "SN_STARTS_WITH_AND": 
            defs.Reason(
                2, 
                "Satellite starts with 'and'."
            ),
        "NUCLEUS_AND_SN_HAVE_SAME_SUBJECT": 
            defs.Reason(
                3, 
                "Nucleus and satellite's nucleus have the same subject."
            ),
        "COMMON_PATTERN_JOINT_JOINT": 
            defs.Reason(
                4,
                "Common pattern (Joint-Joint)."
            ),
        "COMMON_PATTERN_ELABORATION_JOINT": 
            defs.Reason(
                6,
                "Common pattern (Elaboration-Joint)."
            )
    }
    
    def generate_statement(self, text, relation, verbose=False, **kwargs):
        assert(relation is not None and relation.type == "Explanation")
        info = preparation.Preprocessor.prepare_extended_info(text, relation, verbose)
        if info is None:
            utils.print_if_verbose("Extended info preparation wasn't successful.", verbose)
            return None
        
        if info.satellite_info.relation is None:
            return None
        
        reason = None
        
        if utils.contains_any_of(
            utils.remove_extra_space(
                info.satellite_preparation_result.prepared_text.lower()
            ),
            [
                "in fact",
                "as a matter of fact",
                "actually",
                "indeed",
                "also"
            ]
        ):
            reason = self.reasons["SN_CONTAINS_IN_FACT_ETC"]
        elif utils.get_first_token(
            info.satellite_preparation_result.prepared_text
        ) == "and":
            reason = self.reasons["SN_STARTS_WITH_AND"]
        else:
            nucleus_subject = nlp.find_subject(
                text, info.nucleus_info.segment.start, info.nucleus_info.segment.end
            )
            if nucleus_subject is not None:
                utils.print_if_verbose(
                    f"Nucleus' subject is '{nucleus_subject}'.", verbose
                )
                sn_subject = nlp.find_subject(
                    text, info.sn_segment.start, info.sn_segment.end
                )
                if sn_subject is not None:
                    utils.print_if_verbose(
                        f"The subject of satellite's nucleus is '{sn_subject}'.",
                        verbose
                    )
                    if nucleus_subject == sn_subject:
                        reason = self.reasons["NUCLEUS_AND_SN_HAVE_SAME_SUBJECT"]
                else:
                    utils.print_if_verbose(
                        "Failed to find a subject in the satellite's nucleus.", verbose
                    )    
            else:
                utils.print_if_verbose(
                    "Failed to find a subject in the nucleus.", verbose
                )
            
        if reason is None: 
            # -- checking for a common pattern
            utils.print_if_verbose(
                "Nucleus relation type: "
                f"'{utils.get_relation_type(info.nucleus_info.relation)}'.",
                verbose
            )
            utils.print_if_verbose(
                "Satellite relation type: "
                f"'{utils.get_relation_type(info.satellite_info.relation)}'.",
                verbose
            )
            if (
                info.nucleus_info.relation is not None
                    and info.satellite_info.relation is not None
            ):
                if (
                    info.nucleus_info.relation.type == "Elaboration"
                        and info.satellite_info.relation.type == "Joint"
                ):
                    reason = self.reasons["COMMON_PATTERN_ELABORATION_JOINT"]
                elif (
                    info.nucleus_info.relation.type == "Joint"
                        and info.satellite_info.relation.type == "Joint"
                ):
                    reason = self.reasons["COMMON_PATTERN_JOINT_JOINT"]

        if reason is not None:
            utils.print_if_verbose(reason.explanation, verbose)
            final_nucleus_text, final_sn_text = self._finalise_statement_parts(
                info.nucleus_preparation_result.prepared_text,
                info.satellite_preparation_result.prepared_text,
                verbose,
                add_punctuation=True
            )
            return self._generate_statement(
                final_nucleus_text, 
                " Moreover, ", 
                final_sn_text, 
                relation, 
                info.nucleus_proximity,
                self.name,
                reason,
                verbose
            )
        else:
            return None

In [5]:
if __name__ == "__main__" and "__file__" not in globals():
    rule = RuleExplanation03()

    with open("../parsed/race/test/middle/3797.txt.tree", "rt") as f:
        tree_text = f.read()

    text, relations = relation_extraction.read_relations(
        tree_text.replace("<s>", "").replace("<P>", "")
    )

    expl = relations["Explanation"][2]
    print(text[expl.left.start:expl.right.end])

    statement =rule.generate_statement(text, expl, verbose=True)
    print("\nRESULT:")
    if statement is not None:
        print(json.dumps(statement._asdict(), indent=2))

The tiger jumped out .  " How silly you are , " the tiger laughed .  " Now I am going to eat you . "  " Wait ! " the man cried .  " You ought not to eat me .  Let us ask others what they think . "  " You may ask three others .  " the tiger said .  The good man asked a tree .  The tree said , " I give shade .  And yet I am cut down .  Let the tiger eat you . "  Next , the good man asked a bird .  The bird said , " I hurt no one .  
Nucleus is on the left.
Nucleus's depth <= 100.
Nucleus is flat.
Will use the whole segment.
Satellite's nucleus is on the left.
Nuclei proximity is NucleusProximity.NEAR
Satellite's (left) nucleus contains '.', '!', '?', or ';'.
Will use the whole segment.
Text extracted from the satellite:
" How silly you are, " the tiger laughed.  " Now I am going to eat you. "  " Wait! " the man cried.  " You ought not to eat me.  Let us ask others what they think. "  " You may ask three others.  " the tiger said.  The good man asked a tree.  The tree said, " I give shade

In [6]:
if __name__ == "__main__" and "__file__" not in globals():
    rule = RuleExplanation03()

    with open("../parsed/race/train/middle/6287.txt.tree", "rt") as f:
        tree_text = f.read()

    text, relations = relation_extraction.read_relations(
        tree_text.replace("<s>", "").replace("<P>", "")
    )

    expl = relations["Explanation"][0]
    print(text[expl.left.start:expl.right.end])

    statement =rule.generate_statement(text, expl, verbose=True)
    print("\nRESULT:")
    if statement is not None:
        print(json.dumps(statement._asdict(), indent=2))

Students who improve their reading and writing skills will increase their vocabulary and their ability to use the English language correctly .  As parents , you can play an important role in improving your children 's literacy .  You can make a difference by listening , asking questions , giving information , and reading with your children outside school hours .  Here are 8 tips on how to help your children to read and write : ^ having everyday conversations with your children ^ trying to ask more " wh - " questions instead of easy " yes " or " no " questions ^ encouraging your children to talk about events you do n't know ^ reading to your children each night ^ listening to your children read ^ teaching your children how to use dictionaries ^ talking about your children 's writings ^ practicing filling in forms , writing letters , sending cards or emails You can also encourage your children to join the Reading Challenge Club and discuss the books they are reading together .  In the cl